In [1]:
import numpy as np
import random

# generate data

In [2]:
# In real-world scenarios, learning how the data was generated is impractical. Do not rely on this function while doing research.
def generate_data(dim, num):
    x = np.random.normal(0, 10, [num, dim])
    coef = np.random.uniform(-1, 1, [dim, 1])
    pred = np.dot(x, coef)
    pred_n = (pred - np.mean(pred)) / np.sqrt(np.var(pred))
    label = np.sign(pred_n)
    mislabel_value = np.random.uniform(0, 1, num)
    mislabel = 0
    mis_label=[]
    for i in range(num):
        if np.abs(pred_n[i]) < 1 and mislabel_value[i] > 0.9 + 0.1 * np.abs(pred_n[i]):
            label[i] *= -1
            mislabel += 1
            mis_label.append(i)
    return x, label, mislabel/num,mis_label

# write your model class

In [3]:
def accuracy(orin_y,predict_y) :
    len_ = len(orin_y)
    acc = 0
    for i in range(len_) :
        if orin_y[i] == predict_y[i] :
            acc += 1
    return acc/len_

## 软间隔SVM

In [4]:
# Modifying the overall structure is acceptable but not recommended
class SOFT_SVM:
    def __init__(self, dim):
        self.dim=dim
    
    def f(self,alpha,X,y,b,x) :
        sum_=0
        for i in range(len(X)) :
            sum_+=alpha[i]*y[i]*np.dot(X[i],x)
        return sum_+b

    def __SMO(self,X, y, C,tol=1e-5, max_iter=2000):  
        n = self.dim
        m=len(X)
        alpha = np.zeros(m)  
        time = 0
        b = 0
        self.w,self.b=0,0
        pre_acc = 0
        while time < max_iter :
            time += 1
            if time %100==0 :
                print(time,pre_acc)
            i = random.randint(0,m-1)
            while True :
                j = random.randint(0,m-1)
                if j != i : break
            Ei=self.f(alpha,X,y,b,X[i])-y[i]
            Ej=self.f(alpha,X,y,b,X[j])-y[j]
            Kii=np.dot(X[i],X[i])
            Kij=np.dot(X[i],X[j])
            Kjj=np.dot(X[j],X[j])
            eta=Kii+Kjj-2*Kij
            temp_alpha_j=alpha[j]+y[j]*(Ei-Ej)/eta
            old_j,old_i,old_b=alpha[j],alpha[i],self.b
            old_w=1*self.w
            if abs(temp_alpha_j - old_j) < 0.00002:
                continue
            if y[i] == y[j] :
                low = max(0,alpha[j]+alpha[i]-C)
                high = min(C,alpha[i] + alpha[j])
            else :
                low = max(0,alpha[j]-alpha[i])
                high = min(C,C+alpha[j]-alpha[i])
            if temp_alpha_j>high :
                change_alpha_j=high
            elif low<=temp_alpha_j and temp_alpha_j<=high :
                change_alpha_j=temp_alpha_j
            else :
                change_alpha_j=low
            change_alpha_i=alpha[i]+y[i]*y[j]*(alpha[j]-change_alpha_j)
            alpha[i]=change_alpha_i
            alpha[j]=change_alpha_j
            bi = -Ei - y[i]*Kii*(alpha[i] - old_i) - y[j]*Kij*(alpha[j] - old_j) + b
            bj = -Ej - y[i]*Kij*(alpha[i] - old_i) - y[j]*Kjj*(alpha[j] - old_j) + b
            if alpha[i] > 0 and alpha[i]<C:
                b = bi
            elif alpha[j] > 0 and alpha[j]<C:
                b = bj
            else:
                b = (bi + bj)/2
            self.w=np.zeros(n)
            self.b=b
            for k in range(m) :
                self.w += alpha[k]*y[k]*X[k]
            acc=accuracy(self.predict(X),y)
            if pre_acc > acc :
                self.w=1*old_w
                self.b=1*old_b
                alpha[i]=old_i
                alpha[j]=old_j
                continue
            pre_acc=acc
            
    def fit(self, X, y, C): #C为惩罚项系数
        m=len(X)
        self.__SMO(X, y, m, C)
        
    def predict(self, X):
        predict_y=np.zeros(len(X))
        for i in range(len(X)) :
            predict_y[i]=np.dot(X[i],self.w)+self.b
        for i in range(len(X)) :
            if predict_y[i]>=0 :
                predict_y[i] = 1
            else :
                predict_y[i] = -1
        return predict_y

## 硬间隔随机SMO

In [5]:
# Modifying the overall structure is acceptable but not recommended
class RAND_SVM:
    def __init__(self, dim):
        """
        Adding other parameters is acceptable but not necessary.
        """
        self.dim=dim
    
    def f(self,alpha,X,y,b,x) :
        sum_=0
        for i in range(len(X)) :
            sum_+=alpha[i]*y[i]*np.dot(X[i],x)
        return sum_+b

    def __SMO(self,X, y, m,tol=1e-5, max_iter=2000):  
        n = self.dim
        alpha = np.zeros(m)  
        time = 0
        b = 0
        self.w=np.zeros(self.dim)
        self.b=0
        while time < max_iter :
            i = random.randint(0,m-1)
            while True :
                j = random.randint(0,m-1)
                if j != i : break
            Ei=self.f(alpha,X,y,b,X[i])-y[i]
            Ej=self.f(alpha,X,y,b,X[j])-y[j]
            Kii=np.dot(X[i],X[i])
            Kij=np.dot(X[i],X[j])
            Kjj=np.dot(X[j],X[j])
            eta=Kii+Kjj-2*Kij
            temp_alpha_j=alpha[j]+y[j]*(Ei-Ej)/eta
            old_j,old_i=alpha[j],alpha[i]
            if abs(temp_alpha_j - old_j) < 0.00001:
                continue
            if y[i] == y[j] :
                low = 0
                high = alpha[i] + alpha[j]
            else :
                low = max(0,alpha[j]-alpha[i])
                high = np.inf
            if temp_alpha_j>high :
                change_alpha_j=high
            elif low<=temp_alpha_j and temp_alpha_j<=high :
                change_alpha_j=temp_alpha_j
            else :
                change_alpha_j=low
            change_alpha_i=alpha[i]+y[i]*y[j]*(alpha[j]-change_alpha_j)
            alpha[i]=change_alpha_i
            alpha[j]=change_alpha_j
            bi = -Ei - y[i]*Kii*(alpha[i] - old_i) - y[j]*Kij*(alpha[j] - old_j) + b
            bj = -Ej - y[i]*Kij*(alpha[i] - old_i) - y[j]*Kjj*(alpha[j] - old_j) + b
            if alpha[i] > 0 :
                b = bi
            elif alpha[j] > 0:
                b = bj
            else:
                b = (bi + bj)/2
            time += 1
            self.w=np.zeros(n)
            self.b=b
            for k in range(m) :
                self.w += alpha[k]*y[k]*X[k]
            acc=accuracy(self.predict(X),y)
            if time%100==0 :
                print(time,acc)
        w=np.zeros(self.dim)
        for i in range(m) :
            w += alpha[i]*y[i]*X[i]
        self.w=w
        self.b=b

    def fit(self, X, y):
        """
        Fit the coefficients via your method1
        """
        m=len(X)
        self.__SMO(X, y, m)
        
    def predict(self, X):
        predict_y=np.zeros(len(X))
        for i in range(len(X)) :
            predict_y[i]=np.dot(X[i],self.w)+self.b
        for i in range(len(X)) :
            if predict_y[i]>=0 :
                predict_y[i] = 1
            else :
                predict_y[i] = -1
        return predict_y

## 硬间隔SVM启发式SMO 

In [6]:
# Modifying the overall structure is acceptable but not recommended
class SMO_HARD_SVM:
    def __init__(self, dim):
        """
        Adding other parameters is acceptable but not necessary.
        """
        self.dim=dim
    
    def f(self,alpha,X,y,b,x) :
        sum_=0
        for i in range(len(X)) :
            sum_+=alpha[i]*y[i]*np.dot(X[i],x)
        return sum_+b

    def SMO(self,X, y, m,tol=1e-5, max_iter=10):  
        n = self.dim
        alpha = np.ones(m)  
        time = 0
        b = 0
        self.w=np.zeros(n)
        self.b=0
        while time < max_iter :
            con = np.zeros(m)
            min__=np.inf
            min_flag=-1
            for k in range(m) :
                g = y[k][0]*self.f(alpha,X,y,b,X[k])[0]-1
                if g < min__ :
                    min__=g
                    min_flag=k
            i = min_flag
#             计算距KKT最大
            mindist=np.inf
            mindist_flag=-1
            for k in range(m) :
                if i != k :
                    d = np.linalg.norm(X[i]-X[k])
                    if d < mindist :
                        mindist,mindist_flag=d,k
            j = mindist_flag
#             强SMO计算距离X[i]最大
            Ei=self.f(alpha,X,y,b,X[i])-y[i]
            Ej=self.f(alpha,X,y,b,X[j])-y[j]
            Kii=np.dot(X[i],X[i])
            Kij=np.dot(X[i],X[j])
            Kjj=np.dot(X[j],X[j])
            eta=Kii+Kjj-2*Kij
            temp_alpha_j=alpha[j]+y[j]*(Ei-Ej)/eta
            old_j,old_i=alpha[j],alpha[i]
            if y[i] == y[j] :
                low = 0
                high = alpha[i] + alpha[j]
            else :
                low = max(0,alpha[j]-alpha[i])
                high = np.inf
            if temp_alpha_j>high :
                change_alpha_j=high
            elif low<=temp_alpha_j and temp_alpha_j<=high :
                change_alpha_j=temp_alpha_j
            else :
                change_alpha_j=low
            change_alpha_i=alpha[i]+y[i]*y[j]*(alpha[j]-change_alpha_j)
            alpha[i]=change_alpha_i
            alpha[j]=change_alpha_j
            bi = -Ei - y[i]*Kii*(alpha[i] - old_i) - y[j]*Kij*(alpha[j] - old_j) + b
            bj = -Ej - y[i]*Kij*(alpha[i] - old_i) - y[j]*Kjj*(alpha[j] - old_j) + b
            if alpha[i] > 0 :
                b = bi
            elif alpha[j] > 0:
                b = bj
            else:
                b = (bi + bj)/2
            self.w=np.zeros(n)
            for k in range(m) :
                self.w += alpha[k]*y[k]*X[k]
            self.b=b
            print(time,accuracy(self.predict(X),y),i,j)
            time+=1
            if time %1000==0 :
                print(time)

    def fit(self, X, y):
        """
        Fit the coefficients via your method1
        """
        m=len(X)
        self.SMO(X, y, m)
        
    def predict(self, X):
        predict_y=np.zeros(len(X))
        for i in range(len(X)) :
            predict_y[i]=np.dot(X[i],self.w)+self.b
        for i in range(len(X)) :
            if predict_y[i]>=0 :
                predict_y[i] = 1
            else :
                predict_y[i] = -1
        return predict_y
# 如果采用强SMO，其实如果两个标错的数据同时被选中会导致死循环!!!

## SVM随机SMO，但是根据准确率筛除

In [7]:
# Modifying the overall structure is acceptable but not recommended
class SVM1:
    def __init__(self, dim):
        self.dim=dim
    
    def f(self,alpha,X,y,b,x) :
        sum_=0
        for i in range(len(X)) :
            sum_+=alpha[i]*y[i]*np.dot(X[i],x)
        return sum_+b

    def SMO(self,X, y, m,tol=1e-5, max_iter=2000):  
        n = self.dim
        alpha = np.ones(m)  
        time = 0
        b = 0
        pre_acc=0
        self.w=np.zeros(n)
        self.b=0
        while time < max_iter :
            time += 1
            if time %100==0 :
                print(time,pre_acc)
            con = np.zeros(m)
            min__=np.inf
            min_flag=-1
            i = random.randint(0,m-1) 
            while True :
                j = random.randint(0,m-1)
                if j != i : break
            Ei=self.f(alpha,X,y,b,X[i])-y[i]
            Ej=self.f(alpha,X,y,b,X[j])-y[j]
            Kii=np.dot(X[i],X[i])
            Kij=np.dot(X[i],X[j])
            Kjj=np.dot(X[j],X[j])
            eta=Kii+Kjj-2*Kij
            temp_alpha_j=alpha[j]+y[j]*(Ei-Ej)/eta
            old_j,old_i,old_b=alpha[j],alpha[i],self.b
            old_w=1*self.w
            if y[i] == y[j] :
                low = 0
                high = alpha[i] + alpha[j]
            else :
                low = max(0,alpha[j]-alpha[i])
                high = np.inf
            if temp_alpha_j>high :
                change_alpha_j=high
            elif low<=temp_alpha_j and temp_alpha_j<=high :
                change_alpha_j=temp_alpha_j
            else :
                change_alpha_j=low
            change_alpha_i=alpha[i]+y[i]*y[j]*(alpha[j]-change_alpha_j)
            alpha[i]=change_alpha_i
            alpha[j]=change_alpha_j
            bi = -Ei - y[i]*Kii*(alpha[i] - old_i) - y[j]*Kij*(alpha[j] - old_j) + b
            bj = -Ej - y[i]*Kij*(alpha[i] - old_i) - y[j]*Kjj*(alpha[j] - old_j) + b
            if alpha[i] > 0 :
                b = bi
            elif alpha[j] > 0:
                b = bj
            else:
                b = (bi + bj)/2
            self.w=np.zeros(n)
            self.b=b
            for k in range(m) :
                self.w += alpha[k]*y[k]*X[k]
            acc=accuracy(self.predict(X),y)
            if pre_acc > acc :
                self.w=1*old_w
                self.b=1*old_b
                alpha[i]=old_i
                alpha[j]=old_j
                continue
            pre_acc=acc

    def fit(self, X, y):
        """
        Fit the coefficients via your method1
        """
        m=len(X)
        self.SMO(X, y, m)
        
    def predict(self, X):
        predict_y=np.zeros(len(X))
        for i in range(len(X)) :
            predict_y[i]=np.dot(X[i],self.w)+self.b
        for i in range(len(X)) :
            if predict_y[i]>=0 :
                predict_y[i] = 1
            else :
                predict_y[i] = -1
        return predict_y

## 梯度下降SVM

In [8]:
# Modifying the overall structure is acceptable but not recommended
class SVM2:
    def __init__(self, dim):
        self.dim=dim
    
    def f(self,alpha,X,y,b,x) :
        sum_=0
        for i in range(len(X)) :
            sum_+=alpha[i]*y[i]*np.dot(X[i],x)
        return sum_+b
    

    def fit(self, X, y,tol,C,lr):
        """
        Fit the coefficients via your method1
        """
        self.w=np.zeros(self.dim)
        self.b=0
        m=len(X)
        time=0
        grad_w=np.zeros(self.dim)
        while True :
            time+=1
            grad_w=np.copy(self.w)
            grad_b=0
            for i in range(m) :
                temp=-y[i][0]*(np.dot(self.w,X[i])+self.b)
                if temp >= 0 :
                    grad_w+=C*(-y[i][0]*X[i])/(1+np.exp(-temp))
                    grad_b+=C*(-y[i][0])/(1+np.exp(-temp))
                else :
                    grad_w+=C*np.exp(temp)*(-y[i][0]*X[i])/(1+np.exp(temp))
                    grad_b+=C*np.exp(temp)*(-y[i][0])/(1+np.exp(temp))
            self.w-=lr*grad_w
            self.b-=lr*grad_b
            if grad_b**2+np.dot(grad_w,grad_w) < tol :
                break

    def predict(self, X):
        predict_y=np.zeros(len(X))
        for i in range(len(X)) :
            predict_y[i]=np.dot(X[i],self.w)+self.b
        for i in range(len(X)) :
            if predict_y[i]>=0 :
                predict_y[i] = 1
            else :
                predict_y[i] = -1
        return predict_y
            


# construct and train your models

In [23]:
# example
x, y, mr,mis_label = generate_data(20, 1000)

In [39]:
mr

0.036

In [24]:
mis_label #打印出来标错数据

[12,
 64,
 104,
 106,
 177,
 184,
 200,
 218,
 226,
 230,
 321,
 352,
 377,
 408,
 416,
 428,
 440,
 460,
 462,
 474,
 488,
 518,
 528,
 634,
 680,
 712,
 723,
 768,
 776,
 811,
 887,
 896,
 902,
 913,
 957,
 991]

In [25]:
soft_svm=SOFT_SVM(20)
soft_svm.fit(x,y,1)

100 0.796
200 0.797
300 0.801
400 0.868
500 0.868
600 0.875
700 0.883
800 0.886
900 0.891
1000 0.894
1100 0.896
1200 0.896
1300 0.896
1400 0.896
1500 0.896
1600 0.905
1700 0.906
1800 0.907
1900 0.907
2000 0.907


In [26]:
rand_svm=RAND_SVM(20)
rand_svm.fit(x,y)

100 0.725
200 0.885
300 0.858
400 0.802
500 0.893
600 0.829
700 0.907
800 0.776
900 0.736
1000 0.907
1100 0.796
1200 0.843
1300 0.869
1400 0.832
1500 0.868
1600 0.83
1700 0.632
1800 0.901
1900 0.847
2000 0.893


In [27]:
smo_hard_svm=SMO_HARD_SVM(20)
smo_hard_svm.fit(x,y)

0 0.686 991 518
1 0.807 428 94
2 0.658 488 973
3 0.859 218 168
4 0.688 488 973
5 0.754 440 10
6 0.685 488 973
7 0.754 440 10
8 0.685 488 973
9 0.754 440 10


In [28]:
svm1=SVM1(20)
svm1.fit(x,y)

100 0.939
200 0.945
300 0.947
400 0.947
500 0.95
600 0.952
700 0.952
800 0.952
900 0.952
1000 0.952
1100 0.952
1200 0.952
1300 0.952
1400 0.953
1500 0.953
1600 0.953
1700 0.953
1800 0.953
1900 0.953
2000 0.953


In [29]:
svm2=SVM2(20)
svm2.fit(x,y,1e-4,1,0.0001)

# predict and compare your results

In [30]:
# make prediction
# pred = model1.predict()

# compare with generated label

# compare each method

# (Optional) compare with sklearn

In [31]:
soft_predict=soft_svm.predict(x)
accuracy(y,soft_predict)

0.907

In [32]:
rand_svm_predict=rand_svm.predict(x)
accuracy(y,rand_svm_predict)

0.893

In [33]:
smo_hard_svm_predict=smo_hard_svm.predict(x)
accuracy(y,smo_hard_svm_predict)

0.754

In [34]:
svm1_predict=svm1.predict(x)
accuracy(y,svm1_predict)

0.953

In [35]:
svm2_predict=svm2.predict(x)
accuracy(y,svm2_predict)

0.954

In [36]:
from sklearn import datasets  
from sklearn import svm  
from sklearn.model_selection import train_test_split  
from sklearn.metrics import accuracy_score  
  
# 加载数据  
  
# 创建SVM分类器  
clf = svm.SVC(kernel='linear')  # 你也可以试试其他的核函数，比如 'rbf'。  
  
# 训练模型  
clf.fit(x, y)  
  
# 预测测试集  
y_pred = clf.predict(x)  
  
# 计算准确率  
print('Accuracy: ', accuracy_score(y, y_pred))

C:\Users\86181\AppData\Roaming\Python\Python39\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy:  0.953
